In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import numpy as np
import pandas as pd

In [2]:

# Load your CSV file containing video labels and IDs
csv_path = 'features_df.csv'
df = pd.read_csv(csv_path)

# Get the unique classes
unique_classes = df['gloss'].nunique()

print("Number of unique classes:", unique_classes)

Number of unique classes: 2000


In [3]:
# Initialize Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
# Function to extract keypoints from a frame
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,
                              mp_holistic.FACEMESH_CONTOURS)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,
                              mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(
                                  color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(245, 66, 230), thickness=2, circle_radius=2)
                              )

In [7]:
"""
# Load your CSV file containing video labels and IDs
csv_path = 'features_df.csv'
df = pd.read_csv(csv_path)

# Specify the folder where your videos are stored
videos_folder = 'videos'

# Loop through each row in the CSV file
for index, row in df.iterrows():
    video_filename = str(row['video_id']) + '.mp4'  # Append '.mp4' to the filename
    sign_label = row['gloss']  # Assuming 'gloss' is the column with sign labels

    # Construct the full path to the video
    video_path = os.path.join(videos_folder, video_filename)

    # Load video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video was successfully opened
    if not cap.isOpened():
        print(f"Error: Unable to open video '{video_path}'")
        continue  # Skip to the next iteration of the loop

    # Set up Holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Display the frame with landmarks and label
            cv2.putText(image, f"Sign: {sign_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('Video with Keypoints', image)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        # Release the video capture object
        cap.release()

    # Wait for user input before moving to the next video
    cv2.waitKey(0)

# Close OpenCV windows
cv2.destroyAllWindows()"""

'\n# Load your CSV file containing video labels and IDs\ncsv_path = \'features_df.csv\'\ndf = pd.read_csv(csv_path)\n\n# Specify the folder where your videos are stored\nvideos_folder = \'videos\'\n\n# Loop through each row in the CSV file\nfor index, row in df.iterrows():\n    video_filename = str(row[\'video_id\']) + \'.mp4\'  # Append \'.mp4\' to the filename\n    sign_label = row[\'gloss\']  # Assuming \'gloss\' is the column with sign labels\n\n    # Construct the full path to the video\n    video_path = os.path.join(videos_folder, video_filename)\n\n    # Load video file\n    cap = cv2.VideoCapture(video_path)\n\n    # Check if the video was successfully opened\n    if not cap.isOpened():\n        print(f"Error: Unable to open video \'{video_path}\'")\n        continue  # Skip to the next iteration of the loop\n\n    # Set up Holistic model\n    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:\n        while cap.isOpened():\n      

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten(
    ) if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten(
    ) if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten(
    ) if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten(
    ) if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [9]:
# Function to extract keypoints from a video
def extract_keypoints_from_video(video_path, model, sign_label):
    cap = cv2.VideoCapture(video_path)
    keypoints_data = []

    # Set up Holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Extract keypoints
            keypoints = extract_keypoints(results)

            # Append keypoints to the list
            keypoints_data.append(keypoints)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Display the frame with landmarks and label
            cv2.putText(image, f"Sign: {sign_label}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('Video with Keypoints', image)

    cap.release()
    return np.array(keypoints_data)

In [10]:
"""import os
import pandas as pd
from pymongo import MongoClient

# Specify the folder where your videos are stored
videos_folder = 'videos'

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['sign_language_db']
videos_collection = db['videos']

# Load your CSV file containing video labels and IDs
csv_path = 'features_df.csv'
df = pd.read_csv(csv_path)


start = 738

# Loop through each row in the CSV file
for index, row in df.iterrows():
    if index >= start:
        try:
            video_filename = str(row['video_id']) + '.mp4'
            sign_label = row['gloss']
            video_path = os.path.join(videos_folder, video_filename)

            # Load video file and extract keypoints
            keypoints_data = extract_keypoints_from_video(video_path, mp_holistic.Holistic(
                min_detection_confidence=0.5, min_tracking_confidence=0.5), sign_label)

            # Create a MongoDB document
            document = {
                'video_id': str(row['video_id']),
                'label': sign_label,
                'keypoints': keypoints_data.tolist(),  # Convert NumPy array to Python list
            }

            # Insert the document into MongoDB
            videos_collection.insert_one(document)
            print(f"{index}.{sign_label}:Processed {video_filename}")

        except Exception as e:
            print(f"Error processing {video_filename}: {e}")
    else:
        continue"""

'import os\nimport pandas as pd\nfrom pymongo import MongoClient\n\n# Specify the folder where your videos are stored\nvideos_folder = \'videos\'\n\n# Connect to MongoDB\nclient = MongoClient(\'mongodb://localhost:27017/\')\ndb = client[\'sign_language_db\']\nvideos_collection = db[\'videos\']\n\n# Load your CSV file containing video labels and IDs\ncsv_path = \'features_df.csv\'\ndf = pd.read_csv(csv_path)\n\n\nstart = 738\n\n# Loop through each row in the CSV file\nfor index, row in df.iterrows():\n    if index >= start:\n        try:\n            video_filename = str(row[\'video_id\']) + \'.mp4\'\n            sign_label = row[\'gloss\']\n            video_path = os.path.join(videos_folder, video_filename)\n\n            # Load video file and extract keypoints\n            keypoints_data = extract_keypoints_from_video(video_path, mp_holistic.Holistic(\n                min_detection_confidence=0.5, min_tracking_confidence=0.5), sign_label)\n\n            # Create a MongoDB document\n 

In [11]:
import os
import multiprocessing

num_cores = os.cpu_count()

print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 12


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pymongo
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['sign_language_db']
videos_collection = db['videos']

# Retrieve data from MongoDB
cursor = videos_collection.find({})
data = list(cursor)

client.close()

X = [np.array(doc['keypoints']) for doc in data]
y = np.array([doc['label'] for doc in data])

max_length = max(len(seq) for seq in X)

print("Maximum Sequence Length:", max_length)

max_length = max(len(seq) for seq in X)

X_padded = pad_sequences(X, padding='post', maxlen=max_length, dtype='float32')

print("Number of samples in X:", len(X))
print("Number of samples in y:", len(y))

X_train, X_temp, y_train, y_temp = train_test_split(
    X_padded, y, test_size=0.2, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(
    X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val_scaled = scaler.transform(
    X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(
    X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)


print("Shape of scaled training data:", X_train_scaled.shape)
print("Shape of scaled validation data:", X_val_scaled.shape)
print("Shape of scaled test data:", X_test_scaled.shape)

Maximum Sequence Length: 195
Number of samples in X: 894
Number of samples in y: 894


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models


unique_labels = np.unique(y_train)
print("Unique Labels in y_train:", unique_labels)

label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
y_train_encoded = np.array([label_to_int[label] for label in y_train])
y_val_encoded = np.array([label_to_int[label] for label in y_val])
y_test_encoded = np.array([label_to_int[label] for label in y_test])

num_classes = len(unique_labels)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=num_classes)

query = X_train_scaled
value = X_train_scaled


model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(
    X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(X_train_scaled, y_train_one_hot, epochs=10,
          validation_data=(X_val_scaled, y_val_one_hot), batch_size=32)

Unique Labels in y_train: ['accident' 'all' 'apple' 'bed' 'before' 'bird' 'black' 'blue' 'book'
 'bowling' 'can' 'candy' 'chair' 'change' 'clothes' 'color' 'computer'
 'cool' 'corn' 'cousin' 'cow' 'dance' 'dark' 'deaf' 'doctor' 'dog' 'drink'
 'eat' 'enjoy' 'family' 'fine' 'finish' 'fish' 'forget' 'give' 'go'
 'graduate' 'hat' 'hearing' 'help' 'hot' 'kiss' 'language' 'last' 'later'
 'like' 'man' 'many' 'meet' 'mother' 'no' 'now' 'orange' 'pink' 'pizza'
 'play' 'shirt' 'study' 'table' 'tall' 'thanksgiving' 'thin' 'walk' 'what'
 'white' 'who' 'woman' 'wrong' 'year' 'yes']


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 195, 1662)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_one_hot)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
predictions = model.predict(X_test_scaled)

predicted_labels = np.argmax(predictions, axis=1)

int_to_label = {idx: label for label, idx in label_to_int.items()}
decoded_predictions = [int_to_label[pred] for pred in predicted_labels]

In [ ]:
import joblib

# Save the trained model for future use or deployment
model.save('slt_model.h5')
joblib.dump(scaler, 'scaler.joblib')

# Save the label-to-integer mapping for decoding predictions
np.save('label_to_int.npy', label_to_int)

In [ ]:
# Load the saved model and label-to-integer mapping for inference
import numpy as np
loaded_model = keras.models.load_model('slt_model.h5')
loaded_label_to_int = np.load('label_to_int.npy', allow_pickle=True).item()

new_data_array = np.random.rand(195, 1662)

new_data_point = np.expand_dims(new_data_array, axis=0)

new_data_point_2d = new_data_point.reshape(-1, new_data_point.shape[-1])

new_data_scaled = scaler.transform(new_data_point_2d)

new_data_predictions = loaded_model.predict(
    np.expand_dims(new_data_scaled, axis=0))

new_data_predicted_label = np.argmax(new_data_predictions)

new_data_predicted_class = int_to_label[new_data_predicted_label]

print(f"Predicted Class for New Data: {new_data_predicted_class}")


model.save('slt_model.h5')
joblib.dump(scaler, 'scaler.joblib')

# Save the label-to-integer mapping for decoding predictions
np.save('label_to_int.npy', label_to_int)

In [ ]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
import joblib
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Encode labels if they are not already encoded
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


# Assuming X_train_scaled, y_train_encoded, label_encoder are defined

class MyKerasClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.001, lstm_units=128, dropout_rate=0.2, epochs=10, batch_size=32):
        self.learning_rate = learning_rate
        self.lstm_units = lstm_units
        self.dropout_rate = dropout_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            LSTM(self.lstm_units, input_shape=(
                X_train_scaled.shape[1], X_train_scaled.shape[2]), return_sequences=True),
            LSTM(self.lstm_units, return_sequences=True),
            Dense(len(label_encoder.classes_), activation='softmax')
        ])

        optimizer = Adam(learning_rate=self.learning_rate)

        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def fit(self, X, y):
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42)

        history = self.model.fit(
            X_train, y_train,
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_data=(X_val, y_val),
            verbose=0
        )

        return self

    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=1)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)


# Create an instance of MyKerasClassifier
keras_classifier = MyKerasClassifier()

# Define the parameter grid
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'lstm_units': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'epochs': [5, 10, 15],  # You can adjust the number of epochs
    'batch_size': [16, 32, 64]  # You can adjust the batch size
}

# Use GridSearchCV
grid_search = GridSearchCV(estimator=keras_classifier,
                           param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_result = grid_search.fit(X_train_scaled, y_train_encoded)

# Print the best hyperparameters and their corresponding accuracy
print("Best Hyperparameters:", grid_result.best_params_)

# Print the best hyperparameters and their corresponding accuracy
print("Best Hyperparameters:", grid_result.best_params_)
print("Best Accuracy:", grid_result.best_score_)"""

In [ ]:
import numpy as np


def beam_search_decoder(model, input_sequence, beam_width, max_sequence_length):
    # Initialize the beam with the start symbol
    beam = [(model.encode(input_sequence), [model.START_TOKEN], 0.0)]

    # Iteratively expand and prune the beam
    for _ in range(max_sequence_length):
        candidates = []

        # Expand each sequence in the beam
        for encoder_output, partial_sequence, score in beam:
            # Generate next steps
            next_steps = model.decode(encoder_output, partial_sequence)

            # Calculate scores for each next step
            for next_step, next_step_prob in next_steps:
                candidate = (encoder_output, partial_sequence +
                             [next_step], score - np.log(next_step_prob))
                candidates.append(candidate)

        # Prune the candidates and select the top-k
        candidates = sorted(candidates, key=lambda x: x[2])[:beam_width]

        # Check if any sequence in the beam ends with the end symbol
        beam = [c for c in candidates if c[1][-1] != model.END_TOKEN]

    # Select the sequence with the highest score from the final beam
    best_sequence = min(beam, key=lambda x: x[2])[1][1:]

    return best_sequence

# Example usage:
# input_sequence = "..."  # Input sign language sequence
# decoded_sequence = beam_search_decoder(model, input_sequence, beam_width=3, max_sequence_length=50)
# print(decoded_sequence)

In [ ]:
'''import cv2
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import mediapipe as mp
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
final_model = load_model('slt_model.h5')

# Load the scaler
scaler = joblib.load('scaler.joblib')

# Open a connection to the webcam (change the index if you have multiple cameras)
cap = cv2.VideoCapture(0)

# Initialize Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Set the maximum sequence length and feature dimensions
max_length = 195
feature_dimensions = 1662

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        draw_styled_landmarks(image, results)

        # Extract keypoints
        keypoints = extract_keypoints(results)

        print(keypoints)

        # Ensure consistency in the number of features
        if len(keypoints) != feature_dimensions:
            print(
                "Number of features in the new data does not match the expected number.")
            continue

        new_data = scaler.transform(np.array([keypoints]))

        new_data_padded = pad_sequences(
            [new_data], padding='post', maxlen=max_length, dtype='float32')

        # Verify the shape of new_data_padded
        print("Shape of new_data_padded:", new_data_padded.shape)

        # Make predictions using the trained model
        predictions = final_model.predict(new_data_padded)

        # Post-process the predictions (e.g., convert probabilities to labels)
        predicted_label = np.argmax(predictions[0])

        # Decode the integer label to the original label
        predicted_sign = int_to_label[predicted_label]

        print(predicted_sign)

        # Display the predicted sign on the frame
        cv2.putText(frame, f"Prediction: {predicted_sign}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('Real-Time SLT Prediction', frame)

        # Wait for a key event to break the loop
        if cv2.waitKey(1) & 0xFF == 27:  # Use 'Esc' key to exit
            break

# Release the video capture object and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()'''

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import mediapipe as mp
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
final_model = load_model('slt_model.h5')

# Load the scaler
scaler = joblib.load('scaler.joblib')

# Initialize MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Set the maximum sequence length and feature dimensions
max_length = 195
feature_dimensions = 1662

# Function to extract keypoints from a frame


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Function to draw styled landmarks on the image


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))

    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))

    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,
                              mp_holistic.FACEMESH_CONTOURS)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,
                              mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)


    # Draw right hand connections
video_path = '\videos\69241.mp4'
# Open a connection to the video file
# Replace 'your_video.mp4' with the path to your video file
cap = cv2.VideoCapture(video_path)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        draw_styled_landmarks(image, results)

        # Extract keypoints
        keypoints = extract_keypoints(results)

        # Ensure consistency in the number of features
        if len(keypoints) != feature_dimensions:
            print(
                "Number of features in the new data does not match the expected number.")
            continue

        new_data = scaler.transform(np.array([keypoints]))

        new_data_padded = pad_sequences(
            [new_data], padding='post', maxlen=max_length, dtype='float32')

        # Make predictions using the trained model
        predictions = final_model.predict(new_data_padded)

        # Post-process the predictions (e.g., convert probabilities to labels)
        predicted_label = np.argmax(predictions[0])

        # Decode the integer label to the original label
        predicted_sign = int_to_label[predicted_label]

        print(predicted_sign)

        # Display the predicted sign on the frame
        cv2.putText(image, f"Prediction: {predicted_sign}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('Real-Time SLT Prediction', image)

        # Wait for a key event to break the loop
        if cv2.waitKey(1) & 0xFF == 27:  # Use 'Esc' key to exit
            break

# Release the video capture object and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import mediapipe as mp
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

unique_labels = ['accident' 'all' 'apple' 'bed' 'before' 'bird' 'black' 'blue' 'book'
                 'bowling' 'can' 'candy' 'chair' 'change' 'clothes' 'color' 'computer'
                 'cool' 'corn' 'cousin' 'cow' 'dance' 'dark' 'deaf' 'doctor' 'dog' 'drink'
                 'eat' 'enjoy' 'family' 'fine' 'finish' 'fish' 'forget' 'give' 'go'
                 'graduate' 'hat' 'hearing' 'help' 'hot' 'kiss' 'language' 'last' 'later'
                 'like' 'man' 'many' 'meet' 'mother' 'no' 'now' 'orange' 'pink' 'pizza'
                 'play' 'shirt' 'study' 'table' 'tall' 'thanksgiving' 'thin' 'walk' 'what'
                 'white' 'who' 'woman' 'wrong' 'year' 'yes']


# Load your CSV file containing video labels and IDs
csv_path = 'test_data.csv'
df = pd.read_csv(csv_path)

# Specify the folder where your videos are stored
videos_folder = 'videos'

# Loop through each row in the CSV file
for index, row in df.iterrows():

    # Append '.mp4' to the filename
    video_filename = str(row['video_id']) + '.mp4'
    # Assuming 'gloss' is the column with sign labels
    sign_label = row['gloss']

    if sign_label in unique_labels:

        # Construct the full path to the video
        video_path = os.path.join(videos_folder, video_filename)

        # Load video file
        cap = cv2.VideoCapture(video_path)

        # Check if the video was successfully opened
        if not cap.isOpened():
            print(f"Error: Unable to open video '{video_path}'")
            continue  # Skip to the next iteration of the loop

        # Set up Holistic model
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                keypoints = extract_keypoints(results)

                # Ensure consistency in the number of features
                if len(keypoints) != feature_dimensions:
                    print(
                        "Number of features in the new data does not match the expected number.")
                    continue

                # Display the predicted sign on the frame
                cv2.putText(image, f"Prediction: {predicted_sign}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                # Display the frame
                cv2.imshow('Real-Time SLT Prediction', image)

                # Break the loop if 'q' is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release the video capture object
            cap.release()

        # Wait for user input before moving to the next video
        cv2.waitKey(0)

# Close OpenCV windows
cv2.destroyAllWindows()

0 gloss                                                umbrella
video_id                                                60385
url         http://www.aslsearch.com/signs/videos/umbrella...
Name: 0, dtype: object
1 gloss                                                    wall
video_id                                                62196
url         https://s3-us-west-1.amazonaws.com/files.start...
Name: 1, dtype: object
2 gloss                                                    sell
video_id                                                50268
url         https://signstock.blob.core.windows.net/signsc...
Name: 2, dtype: object
3 gloss                                                     and
video_id                                                65067
url         https://aslsignbank.haskins.yale.edu/dictionar...
Name: 3, dtype: object
4 gloss                                                    deny
video_id                                                15548
url         https://signstock.

In [ ]:
'''# Load your CSV file containing video labels and IDs
csv_path = 'test_data.csv'
df = pd.read_csv(csv_path)

# Specify the folder where your videos are stored
videos_folder = 'videos'

# Loop through each row in the CSV file
for index, row in df.iterrows():
    video_filename = str(row['video_id']) + '.mp4'  # Append '.mp4' to the filename
    sign_label = row['gloss']  # Assuming 'gloss' is the column with sign labels

    if sign_label in 
        # Construct the full path to the video
        video_path = os.path.join(videos_folder, video_filename)

        # Load video file
        cap = cv2.VideoCapture(video_path)

        # Check if the video was successfully opened
        if not cap.isOpened():
            print(f"Error: Unable to open video '{video_path}'")
            continue  # Skip to the next iteration of the loop

        # Set up Holistic model
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Display the frame with landmarks and label
                cv2.putText(image, f"Sign: {sign_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.imshow('Video with Keypoints', image)

                # Break the loop if 'q' is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release the video capture object
            cap.release()

        # Wait for user input before moving to the next video
        cv2.waitKey(0)

# Close OpenCV windows
cv2.destroyAllWindows()'''